## Traffic Crash Analysis

### Data importing and pre-processing

In [0]:
import requests
from pyspark.sql import SparkSession

# Make the GET request
resp = requests.get('https://data.cityofchicago.org/resource/85ca-t3if.json?$query=SELECT%20crash_record_id%2C%20crash_date_est_i%2C%20crash_date%2C%20posted_speed_limit%2C%20traffic_control_device%2C%20device_condition%2C%20weather_condition%2C%20lighting_condition%2C%20first_crash_type%2C%20trafficway_type%2C%20lane_cnt%2C%20alignment%2C%20roadway_surface_cond%2C%20road_defect%2C%20report_type%2C%20crash_type%2C%20intersection_related_i%2C%20private_property_i%2C%20hit_and_run_i%2C%20damage%2C%20date_police_notified%2C%20prim_contributory_cause%2C%20sec_contributory_cause%2C%20street_no%2C%20street_direction%2C%20street_name%2C%20beat_of_occurrence%2C%20photos_taken_i%2C%20statements_taken_i%2C%20dooring_i%2C%20work_zone_i%2C%20work_zone_type%2C%20workers_present_i%2C%20num_units%2C%20most_severe_injury%2C%20injuries_total%2C%20injuries_fatal%2C%20injuries_incapacitating%2C%20injuries_non_incapacitating%2C%20injuries_reported_not_evident%2C%20injuries_no_indication%2C%20injuries_unknown%2C%20crash_hour%2C%20crash_day_of_week%2C%20crash_month%2C%20latitude%2C%20longitude%2C%20location%20ORDER%20BY%20crash_date%20DESC%2C%20crash_record_id%20ASC')

# Create a Spark session
spark = SparkSession.builder.appName("SENG550").getOrCreate()

# Create a Spark DataFrame from the response text
df2 = spark.read.json(spark.sparkContext.parallelize([resp.text]))


In [0]:
# Show the DataFrame
df2.show(5)

+------------------+------------------+--------------------+----------------+-----------------+----------+-----------+--------------------+--------------------+-------------+--------------------+--------------------+---------+--------------------+-------------+--------------+-----------------------+----------------------+---------------------------+-----------------------------+--------------+----------------+----------------------+------------+--------------------+--------------------+-------------+--------------------+---------+--------------+------------------+-----------------------+------------------+--------------------+-----------+--------------------+----------------------+------------------+----------------+--------------------+---------+----------------------+--------------------+-----------------+-----------+--------------+-----------------+
|         alignment|beat_of_occurrence|          crash_date|crash_date_est_i|crash_day_of_week|crash_hour|crash_month|     crash_record

### Extract Weather Condition

In [0]:
feature_weather = df2.select("Weather_condition")
# Extract the values from the selected column using rdd.map
rdd_from_weather = feature_weather.rdd.map(lambda row: row[0])

# Display the top 10 records
print("Top 10 Records from Weather condition:")
print(rdd_from_weather.take(10))

Top 10 Records from Weather condition:
['CLEAR', 'CLEAR', 'CLEAR', 'CLEAR', 'CLEAR', 'CLEAR', 'CLEAR', 'CLEAR', 'CLEAR', 'CLEAR']


### Extract Crash Type

In [0]:
feature_crashType = df2.select("Crash_type")
# Extract the values from the selected column using rdd.map
rdd_from_crashType = feature_crashType.rdd.map(lambda row: row[0])

# Display the top 10 records
print("Top 10 Records from Crash Type:")
print(rdd_from_crashType.take(10))

Top 10 Records from Crash Type:
['INJURY AND / OR TOW DUE TO CRASH', 'NO INJURY / DRIVE AWAY', 'NO INJURY / DRIVE AWAY', 'INJURY AND / OR TOW DUE TO CRASH', 'INJURY AND / OR TOW DUE TO CRASH', 'NO INJURY / DRIVE AWAY', 'INJURY AND / OR TOW DUE TO CRASH', 'NO INJURY / DRIVE AWAY', 'INJURY AND / OR TOW DUE TO CRASH', 'NO INJURY / DRIVE AWAY']


### Extract Lighting Condition and Time of Year Crash Occurred

In [0]:
feature_crashDate = df2.select("Crash_date")
# Extract the values from the selected column using rdd.map
rdd_from_crashDate = feature_crashDate.rdd.map(lambda row: row[0])

feature_lighting = df2.select("Lighting_condition")
rdd_from_lighting = feature_lighting.rdd.map(lambda row: row[0]) 

# Display the top 10 records
print("First 10 Records from Crash Date and Time:")
print(rdd_from_crashDate.take(10))irst

print("First 10 Records from Lighting Condition:")
print(rdd_from_lighting.take(10))

Top 10 Records from Crash Date and Time:
['2023-11-22T01:40:00.000', '2023-11-22T00:50:00.000', '2023-11-22T00:20:00.000', '2023-11-22T00:01:00.000', '2023-11-21T23:50:00.000', '2023-11-21T23:46:00.000', '2023-11-21T22:55:00.000', '2023-11-21T21:50:00.000', '2023-11-21T21:48:00.000', '2023-11-21T21:45:00.000']
Top 10 Records from Lighting Condition:
['DARKNESS, LIGHTED ROAD', 'DARKNESS, LIGHTED ROAD', 'DARKNESS', 'DARKNESS, LIGHTED ROAD', 'DARKNESS, LIGHTED ROAD', 'DARKNESS, LIGHTED ROAD', 'DARKNESS, LIGHTED ROAD', 'DARKNESS', 'DARKNESS', 'DARKNESS']


### Extract Longitude and Latitude

In [0]:
feature_longitude = df2.select("Longitude")
# Extract the values from the selected column using rdd.map
rdd_from_longitude = feature_longitude.rdd.map(lambda row: row[0])

feature_latitude = df2.select("Latitude")
rdd_from_latitude = feature_latitude.rdd.map(lambda row: row[0]) 

# Display the first 10 records
print("First 10 Records from Longitude:")
print(rdd_from_longitude.take(10))

print("First 10 Records from Latitude:")
print(rdd_from_latitude.take(10))

First 10 Records from Longitude:
['-87.636344822', '-87.620865974', '-87.722014257', '-87.740129115', '-87.614428813', '-87.596070684', '-87.566154615', '-87.719071202', '-87.662862407', '-87.717587476']
First 10 Records from Latitude:
['41.75633481', '41.801450146', '41.923994137', '41.883079908', '41.727288038', '41.736662284', '41.755271379', '41.745775042', '41.772320473', '41.946613704']


### Create RDD of wanted features

In [0]:
wanted_columns = df2.select("Crash_type","num_units","Weather_condition","Crash_date","Most_severe_injury","Latitude", "Longitude")
wanted_columns.show(1)
rdd_of_features = wanted_columns.rdd.map(lambda row:[row[0],row[1],row[2],row[3],row[4],row[5],row[6]])


+--------------------+---------+-----------------+--------------------+--------------------+------------+-------------+
|          Crash_type|num_units|Weather_condition|          Crash_date|  Most_severe_injury|    Latitude|    Longitude|
+--------------------+---------+-----------------+--------------------+--------------------+------------+-------------+
|INJURY AND / OR T...|        2|            CLEAR|2023-11-23T00:08:...|REPORTED, NOT EVI...|41.757761477|-87.682965832|
+--------------------+---------+-----------------+--------------------+--------------------+------------+-------------+
only showing top 1 row

[['INJURY AND / OR TOW DUE TO CRASH', '2', 'CLEAR', '2023-11-23T00:08:00.000', 'REPORTED, NOT EVIDENT', '41.757761477', '-87.682965832']]


### Remove all rows where the content of one of the fields is unknown

In [0]:
print(rdd_of_features.count())
#row[0] = Crash_type, row[2] = Weather_condition,  row[4]= Most_severe_injury
cleaned_data_rdd = rdd_of_features.filter(lambda row: row[0]!="UNKNOWN"  and row[2]!="UNKNOWN"  and row[4]!="UNKNOWN")
print(cleaned_data_rdd.count())

1000
933


### Apply linear regression

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
File <command-1870660999485449>:8
      6 feature_columns = ["_1", "_2", "_3", "_4"]
      7 assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
----> 8 cleaned_data_df = assembler.transform(cleaned_data_df)
     10 lr = LinearRegression(featuresCol="features", labelCol="Longitude",maxIter=10, regParam=0.3, elasticNetParam=0.8)
     11 #training = cleaned_data_rdd.take(500).toDF(["Crash_type","num_units","Weather_condition","Crash_date","Most_severe_injury","Latitude", "Longitude"])
     12 # Fit the model

File /databricks/spark/python/pyspark/ml/base.py:262, in Transformer.transform(self, dataset, params)
    260         return self.copy(params)._transform(dataset)
    261     else:
--> 262         return self._transform(dataset)
    263 else:
    264     raise TypeError("Params must be a param map but got %s.